# Frequent Pattern Mining in PySpark's MLlib Project

Let's see if you can use the concepts we learned about in the lecture to try out frequent pattern mining techniques on a new dataset!


## Recap:

Spark MLlib provides two algorithms related to frequent pattern mining (FPM): 

1. FP-growth (Frequent Pattern Growth)
2. PrefixSpan 

The distinction is that FP-growth does not use order information in the itemsets, if any, while PrefixSpan is designed for sequential pattern mining where the itemsets are ordered. 

## Data

You own a mall and through membership cards, you have some basic data about your customers like Customer ID, age, gender, annual income and spending score. Spending Score is something you assign to the customer based on your defined parameters like customer behavior and purchasing data.

**Dataset:** Mall_Customers.csv <br>
**Data Source:**  https://www.kaggle.com/vjchoudhary7/customer-segmentation-tutorial-in-python

## Problem statement

As the mall owner, you want to understand the customers who can be easily grouped together so that a strategy can be provided to the marketing team to plan accordingly. 

*Note:* <br>
You may need to transform the data in a way that will be meaningful for your market analysis. Think about how you might group the customers in the this data.

You will also notice that I did not provide any leading questions in this notebook as I usually do. This to provide a bit of a challenge for you as this is the last concept we will be covering before the final project! I hope you enjoy the challenge :)

In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Mining').getOrCreate()

In [21]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.fpm import FPGrowth

In [9]:
df = spark.read.csv('Datasets/Mall_Customers.csv', inferSchema=True, header=True)
df.show(5)

+----------+------+---+------------------+----------------------+
|CustomerID|Gender|Age|Annual Income (k$)|Spending Score (1-100)|
+----------+------+---+------------------+----------------------+
|         1|  Male| 19|                15|                    39|
|         2|  Male| 21|                15|                    81|
|         3|Female| 20|                16|                     6|
|         4|Female| 23|                16|                    77|
|         5|Female| 31|                17|                    40|
+----------+------+---+------------------+----------------------+
only showing top 5 rows



In [7]:
df.printSchema()

root
 |-- CustomerID: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Annual Income (k$): integer (nullable = true)
 |-- Spending Score (1-100): integer (nullable = true)



In [11]:
df = df.withColumnRenamed("Annual Income (k$)",'income').withColumnRenamed("Spending Score (1-100)",'spending_score')
df.show(5)

+----------+------+---+------+--------------+
|CustomerID|Gender|Age|income|spending_score|
+----------+------+---+------+--------------+
|         1|  Male| 19|    15|            39|
|         2|  Male| 21|    15|            81|
|         3|Female| 20|    16|             6|
|         4|Female| 23|    16|            77|
|         5|Female| 31|    17|            40|
+----------+------+---+------+--------------+
only showing top 5 rows



In [12]:
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+----------+------+---+------+--------------+
|CustomerID|Gender|Age|income|spending_score|
+----------+------+---+------+--------------+
|         0|     0|  0|     0|             0|
+----------+------+---+------+--------------+



In [13]:
df.count()

200

In [17]:
grouped_df = df.withColumn('age_group', expr("CASE WHEN Age < 30 THEN 'Under 30' WHEN Age BETWEEN 30 AND 55 THEN '30 to 55' WHEN Age > 50 THEN '50 +' ELSE 'Other' END AS age_group")) \
    .withColumn("income_group",expr("CASE WHEN income < 40 THEN 'Under 40' WHEN income BETWEEN 40 AND 70 THEN '40 - 70' WHEN income > 70 THEN '70 +' ELSE 'Other' END AS income_group")) \
    .withColumn("spending_group",expr("CASE WHEN spending_score < 30 THEN 'Less than 30' WHEN spending_score BETWEEN 30 AND 60 THEN '30 - 60' WHEN spending_score > 60 THEN '60 +' ELSE 'Other' END AS spending_group"))

grouped_df.show(5)

+----------+------+---+------+--------------+---------+------------+--------------+
|CustomerID|Gender|Age|income|spending_score|age_group|income_group|spending_group|
+----------+------+---+------+--------------+---------+------------+--------------+
|         1|  Male| 19|    15|            39| Under 30|    Under 40|       30 - 60|
|         2|  Male| 21|    15|            81| Under 30|    Under 40|          60 +|
|         3|Female| 20|    16|             6| Under 30|    Under 40|  Less than 30|
|         4|Female| 23|    16|            77| Under 30|    Under 40|          60 +|
|         5|Female| 31|    17|            40| 30 to 55|    Under 40|       30 - 60|
+----------+------+---+------+--------------+---------+------------+--------------+
only showing top 5 rows



In [20]:
grouped_df = grouped_df.withColumn("items",array('Gender','age_group', 'income_group','spending_group'))
grouped_df.show(5)

+----------+------+---+------+--------------+---------+------------+--------------+--------------------+
|CustomerID|Gender|Age|income|spending_score|age_group|income_group|spending_group|               items|
+----------+------+---+------+--------------+---------+------------+--------------+--------------------+
|         1|  Male| 19|    15|            39| Under 30|    Under 40|       30 - 60|[Male, Under 30, ...|
|         2|  Male| 21|    15|            81| Under 30|    Under 40|          60 +|[Male, Under 30, ...|
|         3|Female| 20|    16|             6| Under 30|    Under 40|  Less than 30|[Female, Under 30...|
|         4|Female| 23|    16|            77| Under 30|    Under 40|          60 +|[Female, Under 30...|
|         5|Female| 31|    17|            40| 30 to 55|    Under 40|       30 - 60|[Female, 30 to 55...|
+----------+------+---+------+--------------+---------+------------+--------------+--------------------+
only showing top 5 rows



In [23]:
fpGrowth = FPGrowth(itemsCol='items', minSupport=0.2, minConfidence=0.1)
model = fpGrowth.fit(grouped_df)

In [34]:
popularity = model.freqItemsets
popularity.createOrReplaceTempView('popularity')

In [35]:
spark.sql("SELECT * FROM popularity ORDER BY freq DESC").limit(20).show()

+--------------------+----+
|               items|freq|
+--------------------+----+
|          [30 to 55]| 116|
|            [Female]| 112|
|           [30 - 60]|  92|
|              [Male]|  88|
|           [40 - 70]|  80|
|  [40 - 70, 30 - 60]|  77|
|              [70 +]|  74|
|  [Female, 30 to 55]|  72|
|              [60 +]|  62|
|   [30 - 60, Female]|  55|
|    [70 +, 30 to 55]|  55|
|          [Under 30]|  55|
|   [40 - 70, Female]|  47|
|      [Less than 30]|  46|
| [30 - 60, 30 to 55]|  46|
|          [Under 40]|  46|
|[40 - 70, 30 - 60...|  45|
|    [Male, 30 to 55]|  44|
+--------------------+----+



In [36]:
assoc = model.associationRules
assoc.createOrReplaceTempView('assoc')

In [37]:
spark.sql("SELECT * FROM assoc ORDER BY confidence DESC").limit(20).show()

+------------------+----------+-------------------+------------------+
|        antecedent|consequent|         confidence|              lift|
+------------------+----------+-------------------+------------------+
|         [40 - 70]| [30 - 60]|             0.9625| 2.092391304347826|
| [40 - 70, Female]| [30 - 60]| 0.9574468085106383|2.0814061054579094|
|         [30 - 60]| [40 - 70]| 0.8369565217391305| 2.092391304347826|
| [30 - 60, Female]| [40 - 70]| 0.8181818181818182|2.0454545454545454|
|            [70 +]|[30 to 55]| 0.7432432432432432|1.2814538676607643|
|          [Female]|[30 to 55]| 0.6428571428571429|1.1083743842364533|
|        [30 to 55]|  [Female]| 0.6206896551724138| 1.108374384236453|
|         [30 - 60]|  [Female]| 0.5978260869565217|1.0675465838509315|
|         [40 - 70]|  [Female]|             0.5875|1.0491071428571428|
|[40 - 70, 30 - 60]|  [Female]| 0.5844155844155844| 1.043599257884972|
|         [30 - 60]|[30 to 55]|                0.5|0.8620689655172414|
|     